In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer

In [2]:
# 데이터 불러오기
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')

# 데이터 확인
train_df.head()
test_df.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
0,0.257178,-0.023285,-0.014654,-0.938404,-0.920091,-0.667683,-0.952501,-0.925249,-0.674302,-0.894088,...,-0.705974,0.006462,0.162920,-0.825886,0.271151,-0.720009,0.276801,-0.057978,2,STANDING
1,0.286027,-0.013163,-0.119083,-0.975415,-0.967458,-0.944958,-0.986799,-0.968401,-0.945823,-0.894088,...,-0.594944,-0.083495,0.017500,-0.434375,0.920593,-0.698091,0.281343,-0.083898,2,STANDING
2,0.275485,-0.026050,-0.118152,-0.993819,-0.969926,-0.962748,-0.994403,-0.970735,-0.963483,-0.939260,...,-0.640736,-0.034956,0.202302,0.064103,0.145068,-0.702771,0.280083,-0.079346,2,STANDING
3,0.270298,-0.032614,-0.117520,-0.994743,-0.973268,-0.967091,-0.995274,-0.974471,-0.968897,-0.938610,...,-0.736124,-0.017067,0.154438,0.340134,0.296407,-0.698954,0.284114,-0.077108,2,STANDING
4,0.274833,-0.027848,-0.129527,-0.993852,-0.967445,-0.978295,-0.994111,-0.965953,-0.977346,-0.938610,...,-0.846595,-0.002223,-0.040046,0.736715,-0.118545,-0.692245,0.290722,-0.073857,2,STANDING


In [10]:
# 결측치 확인
train_df.isnull().sum()

# 데이터 분할
X_train = train_df.iloc[:, :-2].values
y_train = train_df.iloc[:, -1].values
X_test = test_df.iloc[:, :-2].values
y_test = test_df.iloc[:, -1].values

# 결측치 처리
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(X_train)
X_train = imputer.transform(X_train)
X_test = imputer.transform(X_test)

# 표준화
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
# 독립변수, 종속변수 설정
X = X_train
y = y_train
print("독립변수 X : ", X)
print("종속변수 Y : ", y)

독립변수 X :  [[ 0.20064157 -0.0636826  -0.41962845 ... -0.68721921  0.40794614
  -0.00756789]
 [ 0.05594788  0.03148567 -0.25390836 ... -0.694138    0.40911698
   0.00787517]
 [ 0.07351535 -0.04341648 -0.07629468 ... -0.702239    0.4102883
   0.02650234]
 ...
 [-0.01566765  0.0167814   1.13222107 ... -0.56584847  0.64059683
   0.34870928]
 [ 0.21586648 -0.02812252 -0.86770988 ... -0.57766781  0.63147758
   0.29327564]
 [ 1.09620157  0.12919873 -1.67268082 ... -0.57392691  0.63274259
   0.33396081]]
종속변수 Y :  [ 1  1  1 ... 30 30 30]


In [12]:
# XGBoost 모델 생성
model = xgb.XGBClassifier()

# 모델 학습
model.fit(X, y)

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20], got [ 1  3  5  6  7  8 11 14 15 16 17 19 21 22 23 25 26 27 28 29 30]

In [ ]:
model.fit(X, y)
y_pred = model.predict(X_test)
print("Validation Accuracy:", accuracy_score(X_test, y_pred))

In [5]:
# 3. 모델 평가 및 예측
test_data = pd.read_csv('test.csv')
X_test = test_data.drop(['Activity', 'subject'], axis=1)
y_test = pd.get_dummies(test_data['Activity'])
X_test = scaler.transform(X_test)
y_pred = model.predict(X_test)
print("Test Accuracy:", accuracy_score(y_test, y_pred))

Test Accuracy: 0.8934509670851714


모델의 예측 결과인 y_pred를 기반으로 패턴 밖의 데이터가 들어왔을 때 알림

새로운 데이터를 로드하고 전처리. 이전에 StandardScaler를 사용하여 데이터를 스케일링한 것과 같은 방법으로 스케일링을 해야함.

model.predict() 함수를 사용하여 새로운 데이터의 예측값을 계산.

예측값이 이전에 학습한 클래스 레이블 중 어느 하나와 다르다면, 패턴 밖의 데이터가 들어왔다는 것. 이때 알림을 보내도록 코드를 작성.

In [ ]:
new_data = pd.read_csv('new_data.csv')  # 새로운 데이터를 로드합니다.
new_data = scaler.transform(new_data.drop(['Activity', 'subject'], axis=1))  # 이전에 사용한 스케일러를 사용하여 데이터를 스케일링합니다.
y_new_pred = model.predict(new_data)  # 새로운 데이터의 예측값을 계산합니다.

if not any((y_new_pred == y_train).all(axis=1)):  # 예측값이 이전에 학습한 클래스 레이블 중 어느 하나와 다르다면
    send_alert()  # 알림을 보냅니다.


In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

# 1. 데이터 로드 및 전처리
data = pd.read_csv('train.csv')
X = data.drop(['Activity', 'subject'], axis=1)
y = pd.get_dummies(data['Activity'])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# 2. XGBoost 모델 훈련
model = xgb.XGBClassifier()
model.fit(X_train, y_train)

# 3. 모델 평가 및 예측
test_data = pd.read_csv('test.csv')
X_test = test_data.drop(['Activity', 'subject'], axis=1)
y_test = pd.get_dummies(test_data['Activity'])
X_test = scaler.transform(X_test)
y_pred = model.predict(X_test)

# 4. 모델 예측 결과 평가
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)

# 5. 패턴 밖의 데이터 예측
new_data = pd.read_csv('new_data.csv')
X_new = new_data.drop(['Activity', 'subject'], axis=1)
y_new = pd.get_dummies(new_data['Activity'])
X_new = scaler.transform(X_new)
y_pred_new = model.predict(X_new)

# 6. 패턴 밖의 데이터 예측 결과 평가 및 알림
accuracy_new = accuracy_score(y_new, y_pred_new)
print("New Data Accuracy:", accuracy_new)

threshold = 0.05  # 일정 오차 범위 설정
diff = np.abs(y_new.values.argmax(axis=1) - y_pred_new.argmax(axis=1))
if np.max(diff) <= threshold:
    print("예측 결과가 정확합니다.")
else:
    print("경고: 예측 결과가 오차 범위를 초과합니다!")
